# HearHere
## Measure local reputation of global business

### 1. Choose company and extract locations

#### 1.1 'BBVA' offices extractor

In [1]:
import pandas as pd
import numpy as np
import json

from config import data_path

In [2]:
#Setup functions requests and path
update_bbva_offices = False
bbva_offices_path = data_path + 'bbva_offices.csv'

#Load data
if update_bbva_offices: 
    
    #Import BBVA offices scraper
    from scrapers import bbva_extract_offices
    
    #Export to .csv
    bbva_offices = bbva_extract_offices()
    bbva_offices.to_csv(bbva_offices_path, index=False)
    
else:
    bbva_offices = pd.read_csv(bbva_offices_path)
    
#Drop unneeded columns
bbva_offices = bbva_offices.drop(columns=['office_url', 'municipality_url', 'province_url'])

#Drop empty rows
bbva_offices.replace('', np.nan, inplace=True)
bbva_offices = bbva_offices.dropna()
    
bbva_offices.head()

,office_number,office_direction,municipality_name,province_name
0,92.0,"Adra - Natalio Rivas, 23, C.P:4770, Adra, Alm...",Adra,Almería
1,5764.0,"Aguadulce - Av. Carlos Iii Esq. C/lima, C.P:47...",Aguadulce,Almería
2,5461.0,"Almeria - Empresas - Paseo Almeria, 41, C.P:40...",Almeria,Almería
3,1809.0,"Almeria - Ps. Almeria 41 - Ps. Almeria, 41, C....",Almeria,Almería
4,6828.0,Almeria - Edf Oliveros - Canonigo Molina Alons...,Almeria,Almería


#### 1.2 Improve 'office_direction' data

In [3]:
#Remove the first municipality appearance in 'office_direction'
bbva_offices['office_direction'] = bbva_offices['office_direction'].replace({r'^.+? - ': ''}, regex=True)
bbva_offices.head()

,office_number,office_direction,municipality_name,province_name
0,92.0,"Natalio Rivas, 23, C.P:4770, Adra, Almería",Adra,Almería
1,5764.0,"Av. Carlos Iii Esq. C/lima, C.P:4720, Aguadulc...",Aguadulce,Almería
2,5461.0,"Empresas - Paseo Almeria, 41, C.P:4001, Almeri...",Almeria,Almería
3,1809.0,"Ps. Almeria 41 - Ps. Almeria, 41, C.P:4001, Al...",Almeria,Almería
4,6828.0,"Edf Oliveros - Canonigo Molina Alonso, 6, C.P:...",Almeria,Almería


In [4]:
#Extract postal code
bbva_offices['postal_code'] = bbva_offices['office_direction'].str.extract(pat = 'C\.P:(\d*)', expand=True)
bbva_offices.head()

,office_number,office_direction,municipality_name,province_name,postal_code
0,92.0,"Natalio Rivas, 23, C.P:4770, Adra, Almería",Adra,Almería,4770
1,5764.0,"Av. Carlos Iii Esq. C/lima, C.P:4720, Aguadulc...",Aguadulce,Almería,4720
2,5461.0,"Empresas - Paseo Almeria, 41, C.P:4001, Almeri...",Almeria,Almería,4001
3,1809.0,"Ps. Almeria 41 - Ps. Almeria, 41, C.P:4001, Al...",Almeria,Almería,4001
4,6828.0,"Edf Oliveros - Canonigo Molina Alonso, 6, C.P:...",Almeria,Almería,4004


In [5]:
#Remove postal code, province and municipality from 'office_direction'
bbva_offices['office_direction'] = bbva_offices['office_direction'].replace({r', C\.P:.*': ''}, regex=True)
bbva_offices.head()

,office_number,office_direction,municipality_name,province_name,postal_code
0,92.0,"Natalio Rivas, 23",Adra,Almería,4770
1,5764.0,Av. Carlos Iii Esq. C/lima,Aguadulce,Almería,4720
2,5461.0,"Empresas - Paseo Almeria, 41",Almeria,Almería,4001
3,1809.0,"Ps. Almeria 41 - Ps. Almeria, 41",Almeria,Almería,4001
4,6828.0,"Edf Oliveros - Canonigo Molina Alonso, 6",Almeria,Almería,4004


In [6]:
a = bbva_offices['office_direction'].unique()
for x in a:
    print (x)

Natalio Rivas, 23
Av. Carlos Iii Esq. C/lima
Empresas - Paseo Almeria, 41
Ps. Almeria 41 - Ps. Almeria, 41
Edf Oliveros - Canonigo Molina Alonso, 6
Nueva Andalucia - Arcipreste De Hita, 2 Esq. Santiago
Carrera Del Peru - Manuel Azaña, 7 - 8
Zapillo - Av. Cabo De Gata, 107
Los Angeles - Granada, 213
Av.del Mediterraneo - Av.mediterraneo, S/n. Ed.sevilla L.1
La Cañada - Ctra.nijar Km. 4,3 - Cañada San Urbano
El Alquian - Carretera De Nijar, 55 - El Alquian
Zz Campamento Alvarez Sotomayor
Av. Manuel Salmeron, 107
Sorbas, 8
Avda. De Barcelona, 41
Crt Malaga - Carretera De Malaga, 272
Santa Maria Del Aguila - Pº Sta Maria Del Aguila, 67
Mayor - Mayor, 206
Antonio Beltran, 7
Av. De Europa, 195
Bulevard Ciudad De Vicar, 1254
Juan Carlos I - Juan Carlos I, 2
El Parador - S.isidro Labrador,esq.av.fabriquilla
San Jose - San Jose, 16
Av. Constitucion, S/n
Del Mar - Esq.alvarez De Sotomayor L - 8
Virgen Del Carmen, 17 - Virgen Del Carmen, 17
Jose Antonio, 3 - Radio Algeciras, 3
Mezquita - Mezquita

Empresas - Av.del Pais Valencia, 40/42 Planta 1
Alameda - Ps. Alameda, 54
Juan Xxiii - Sabadell, 16
El Albir - Av. D'albir, 2
Garcia Lorca - Federico Garcia Lorca, 33
Rambla - Rambla Mendez Nuñez, 42
Empresas - Rambla Mendez Nuñez, 42 Pl. 1ª
Reyes Catolicos - Reyes Catolicos, 22
O'donnell - General O'donnell, 11
Eusebio Sempere - Av. Eusebio Sempere, 13
Plaza De España - Av. Alcoy, 3
Alfonso El Sabio, 12 - Alfonso X El Sabio, 12
San Blas - Gl. Poeta Vicente Mojica, 7
Av. Estacion - Av. Estacion, 8
Av.orihuela 136 - Av. Orihuela, 136 - 138
Av. Aguilera - Av. Aguilera, 29
Gran Via - Orion, 20
San Gabriel - Ramon Gomez Sempere, 8
Av. De Novelda 59 - Av. De Novelda, 59
Av. Doctor Jimenez Diaz, 7
Maestro Alonso - Maestro Alonso, 148
Padre Espla 2 - Padre Espla, 2
Av. Juan Sanchis Candela - Av. Juan Sanchis Candela, 28
Mirian Blasco - Deportista Kiko Sanchez, 10
Villafranqueza - Practicante Vicente Blasco, 1
Playa San Juan(cos.blanca) - Av. Costa Blanca, 117
Pz. De La Constitucion, 14
Jaime 

#### 2. API Configuration

In [7]:
#Import Google Maps API key and setup the library
from config import geocoding_api_key
if geocoding_api_key == '':
    from api_keys import geocoding_api_key

import googlemaps    
gmaps = googlemaps.Client(key=geocoding_api_key)

#Prueba
geocode_result = gmaps.geocode('BBVA Crt Malaga - Carretera De Malaga, 272')

In [8]:
#Access to the first result of the API if there are more
geocode_result = geocode_result[0]
geocode_result

{'address_components': [{'long_name': 'Edif',
   'short_name': 'Edif',
   'types': ['establishment', 'point_of_interest']},
  {'long_name': '74', 'short_name': '74', 'types': ['street_number']},
  {'long_name': 'Carretera Coín',
   'short_name': 'Ctra. Coín',
   'types': ['route']},
  {'long_name': 'Málaga',
   'short_name': 'Málaga',
   'types': ['locality', 'political']},
  {'long_name': 'Málaga',
   'short_name': 'Málaga',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Andalucía',
   'short_name': 'AL',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'Spain',
   'short_name': 'ES',
   'types': ['country', 'political']},
  {'long_name': '29140', 'short_name': '29140', 'types': ['postal_code']}],
 'formatted_address': 'Edif, Ctra. Coín, 74, 29140 Málaga, Spain',
 'geometry': {'location': {'lat': 36.6616762, 'lng': -4.504492},
  'location_type': 'ROOFTOP',
  'viewport': {'northeast': {'lat': 36.6630251802915,
    'lng': -4.5031430

In [10]:
geocode_result['formatted_address']

'Edif, Ctra. Coín, 74, 29140 Málaga, Spain'

In [13]:
geocode_result['geometry']['location']['lat']

36.6616762

In [18]:
geocode_result['geometry']['location']['lng']

-4.504492

In [20]:
geocode_result['place_id']

'ChIJff1DHEn6cg0RtiT4RfwsnwY'